기본 세팅

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install konlpy
!pip install kss

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 3.8MB 46.5MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251531 sha256=2a4746ada4cc13caa0515690863f496f6a206ffb51df52d79dbb2f08d7fcd415
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss


depression score model

In [ ]:
import pandas as pd
import numpy as np
import kss
import re
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import KFold

def convert_to_ord(data):
    try:
        return [ord(xx) for xx in data]
    except:
        print(data)

df_blog = pd.read_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/blog text data.csv')
df = []

model_yn = load_model('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/model_yn.h5')
model_09 = load_model('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/model_09.h5')

for i in range(len(df_blog)):
    sentence_list = kss.split_sentences(df_blog.iloc[i,3])
    for j in range(len(sentence_list)):
        df.append({'이름':df_blog.iloc[i,0], '날짜':df_blog.iloc[i,1], '제목':df_blog.iloc[i,2], '내용':' '.join(re.sub(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]', ' ', str(sentence_list[j]).strip()).split()),'ord':np.nan,'label_1':np.nan,'label_2':np.nan})

df = pd.DataFrame(df)

df['ord'] = df['내용'].map(convert_to_ord)

max_len = 64

sentence_ord = sequence.pad_sequences(df['ord'], maxlen = max_len)
sentence_ord = np.array(sentence_ord).reshape((len(sentence_ord), max_len))

df['label_1'] = model_yn.predict_classes(sentence_ord)
df['label_2'] = model_09.predict_classes(sentence_ord)

df['label_2'][df['label_1']==0] = np.nan

df_yn = df.groupby(['이름','label_1']).count()
df_yn = df_yn.reset_index()
df_yn = df_yn.pivot(index='이름',columns='label_1',values='내용')
df_yn.columns = ['N','Y']
df_yn['all'] = df_yn['N']+df_yn['Y']
df_yn['p'] = df_yn['Y']/(df_yn['N']+df_yn['Y'])
df_yn = df_yn[['all','p','N','Y']]

df_09 = df.groupby(['이름','label_1','label_2']).count()
df_09 = df_09.reset_index()
df_09 = df_09.pivot(index='이름',columns='label_2',values='내용')
df_09.columns = ['0','1','2','3','4','5','6','7','8','9']

df_score_board = pd.merge(df_yn, df_09, on='이름')

df_real_score = pd.read_excel('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/list.xlsx', encoding='cp949')

df_real_score = df_real_score.replace('없음', 0)
df_real_score = df_real_score.replace('2~6일', 1)
df_real_score = df_real_score.replace('7~12일', 2)
df_real_score = df_real_score.replace('거의 매일', 3)

df_score_board = pd.merge(df_score_board, df_real_score, on='이름')
df_score_board['base_model'] = np.nan
df_score_board['a'] = np.nan
df_score_board['b,c'] = np.nan
df_score_board = df_score_board.fillna(0)

k = 5

kf = KFold(n_splits=k, shuffle=True)
n = 1
for i,j in kf.split(df_score_board):
    df_score_board.iloc[i].to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_train_'+str(n)+'.csv', encoding='cp949', index=False)
    df_score_board.iloc[j].to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_test_'+str(n)+'.csv', encoding='cp949', index=False)
    n += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Baseline model

In [ ]:
import pandas as pd

for n in range(1,k+1,1):
    df_score_board_train = pd.read_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_train_'+str(n)+'.csv', encoding='cp949')
    df_score_board_test = pd.read_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_test_'+str(n)+'.csv', encoding='cp949')

    for i in range(len(df_score_board_train)):
        sum = 0
        for j in range(9):
            if df_score_board_train.iloc[i,6+j] == 0 or df_score_board_train.iloc[i,6+j] == 1:
                sum += 0
            elif df_score_board_train.iloc[i,6+j] == 2 or df_score_board_train.iloc[i,6+j] == 3 or df_score_board_train.iloc[i,6+j] == 4 or df_score_board_train.iloc[i,6+j] == 5 or df_score_board_train.iloc[i,6+j] == 6:
                sum += 1
            elif df_score_board_train.iloc[i,6+j] == 7 or df_score_board_train.iloc[i,6+j] == 8 or df_score_board_train.iloc[i,6+j] == 9 or df_score_board_train.iloc[i,6+j] == 10 or df_score_board_train.iloc[i,6+j] == 11 or df_score_board_train.iloc[i,6] == 12:
                sum += 2
            elif df_score_board_train.iloc[i,6+j] == 13 or df_score_board_train.iloc[i,6+j] == 14:
                sum += 3
        df_score_board_train.iloc[i,36] = sum

    for i in range(len(df_score_board_test)):
        sum = 0
        for j in range(9):
            if df_score_board_test.iloc[i,6+j] == 0 or df_score_board_test.iloc[i,6+j] == 1:
                sum += 0
            elif df_score_board_test.iloc[i,6+j] == 2 or df_score_board_test.iloc[i,6+j] == 3 or df_score_board_test.iloc[i,6+j] == 4 or df_score_board_test.iloc[i,6+j] == 5 or df_score_board_test.iloc[i,6+j] == 6:
                sum += 1
            elif df_score_board_test.iloc[i,6+j] == 7 or df_score_board_test.iloc[i,6+j] == 8 or df_score_board_test.iloc[i,6+j] == 9 or df_score_board_test.iloc[i,6+j] == 10 or df_score_board_test.iloc[i,6+j] == 11 or df_score_board_test.iloc[i,6] == 12:
                sum += 2
            elif df_score_board_test.iloc[i,6+j] == 13 or df_score_board_test.iloc[i,6+j] == 14:
                sum += 3
        df_score_board_test.iloc[i,36] = sum

    df_score_board_train.to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_train_'+str(n)+'.csv', encoding='cp949', index=False)
    df_score_board_test.to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_test_'+str(n)+'.csv', encoding='cp949', index=False)

a

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_absolute_error

for n in range(1,k+1,1):
    df_score_board_train = pd.read_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_train_'+str(n)+'.csv', encoding='cp949')
    df_score_board_test = pd.read_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_test_'+str(n)+'.csv', encoding='cp949')

    A_start = 0
    A_end = 100
    A_gap = 0.01
    A = [np.arange(A_start,A_end,A_gap) for i in range(10)]
    A[0] = 0

    for i in range(1,10,1):
        for j in range(len(A[i])):
            real_score = df_score_board_train.iloc[:,24+i]
            pred_score = [3*math.tanh(A[i][j]*k) for k in df_score_board_train[str(i)]/df_score_board_train['all']]
            A[i][j] = mean_absolute_error(real_score, pred_score)

    for i in range(1,10,1):
        print('a_'+str(i)+' : ', A_start + A_gap*list(A[i]).index(min(A[i])))
        print('a_'+str(i)+'_MSE : ',min(A[i]))
        A[i] = A_start + A_gap*list(A[i]).index(min(A[i]))
    print('========================================')

    for i in range(len(df_score_board_train)):
        df_score_board_train.iloc[i,37] = 3*math.tanh(A[1]*df_score_board_train.iloc[i,6]/df_score_board_train.iloc[i,1]) + 3*math.tanh(A[2]*df_score_board_train.iloc[i,7]/df_score_board_train.iloc[i,1]) + 3*math.tanh(A[3]*df_score_board_train.iloc[i,8]/df_score_board_train.iloc[i,1]) + 3*math.tanh(A[4]*df_score_board_train.iloc[i,9]/df_score_board_train.iloc[i,1]) + 3*math.tanh(A[5]*df_score_board_train.iloc[i,10]/df_score_board_train.iloc[i,1]) + 3*math.tanh(A[6]*df_score_board_train.iloc[i,11]/df_score_board_train.iloc[i,1]) + 3*math.tanh(A[7]*df_score_board_train.iloc[i,12]/df_score_board_train.iloc[i,1]) + 3*math.tanh(A[8]*df_score_board_train.iloc[i,13]/df_score_board_train.iloc[i,1]) + 3*math.tanh(A[9]*df_score_board_train.iloc[i,14]/df_score_board_train.iloc[i,1])

    for i in range(len(df_score_board_test)):
        df_score_board_test.iloc[i,37] = 3*math.tanh(A[1]*df_score_board_test.iloc[i,6]/df_score_board_test.iloc[i,1]) + 3*math.tanh(A[2]*df_score_board_test.iloc[i,7]/df_score_board_test.iloc[i,1]) + 3*math.tanh(A[3]*df_score_board_test.iloc[i,8]/df_score_board_test.iloc[i,1]) + 3*math.tanh(A[4]*df_score_board_test.iloc[i,9]/df_score_board_test.iloc[i,1]) + 3*math.tanh(A[5]*df_score_board_test.iloc[i,10]/df_score_board_test.iloc[i,1]) + 3*math.tanh(A[6]*df_score_board_test.iloc[i,11]/df_score_board_test.iloc[i,1]) + 3*math.tanh(A[7]*df_score_board_test.iloc[i,12]/df_score_board_test.iloc[i,1]) + 3*math.tanh(A[8]*df_score_board_test.iloc[i,13]/df_score_board_test.iloc[i,1]) + 3*math.tanh(A[9]*df_score_board_test.iloc[i,14]/df_score_board_test.iloc[i,1])

    df_score_board_train.to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_train_'+str(n)+'.csv', encoding='cp949', index=False)
    df_score_board_test.to_csv('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_score_model_test_'+str(n)+'.csv', encoding='cp949', index=False)